In [1]:
from utils_dfs import *
from utils_fta import *
import pandas as pd

In [2]:
bench_filepath = "./bench_files/s2.bench"
fault_table_path = "./fault_tables/s2.html"

# Process Circuit and Tests

In [3]:
circuit_data = process_circuit_file(bench_filepath)
input_list, gate_list, fanout_list, output_list = split_circuit(circuit_data)
wires = create_wires(circuit_data)
input_wires = create_input_wires(input_list)
gates_output_wires = create_gates_output_wires(gate_list)

In [4]:
input_values = generate_bit_combinations(n=len(input_list))

In [5]:
U_set = create_union_set(wires)
all_faults = list(U_set)
all_faults = sorted(all_faults, key=fault_sorter)

# Fault Simulation

In [6]:
all_discovered_faults = []
for input_value in input_values:
    input_vector = {key: value for key, value in zip(input_list, list(input_value))}

    simulation_dict = create_simulation_list(wires)
    tvs = true_value_simulation(
        input_vector,
        simulation_dict,
        fanout_list,
        gate_list,
        input_wires,
        gates_output_wires
    )

    La_dict = init_fault_list(wires, tvs)
    dfs = deductive_fault_simulation(
        La_dict,
        input_list,
        fanout_list,
        gates_output_wires,
        gate_list,
        tvs,
        U_set
    )

    discovered_faults = set()
    for output_wire in output_list:
        faults = dfs['L_w' + output_wire]
        discovered_faults = discovered_faults.union(faults)
    all_discovered_faults.append(list(discovered_faults))

# Create Fault Table

In [7]:
fault_table = create_fault_table(input_values, all_discovered_faults, all_faults)

In [8]:
df = pd.DataFrame(fault_table[1:], columns=fault_table[0])
df.to_html(fault_table_path, index=False)
df

,-,w1_s0,w1_s1,w2_s0,w2_s1,w3_s0,w3_s1,w4_s0,w4_s1,w5_s0,...,w5_1_s0,w5_1_s1,w5_2_s0,w5_2_s1,w6_s0,w6_s1,w7_s0,w7_s1,w8_s0,w8_s1
0,0000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0001,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
2,0010,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0011,0,1,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,0100,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0101,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
6,0110,0,0,1,0,1,0,0,0,1,...,1,0,1,0,1,0,1,0,1,0
7,0111,0,0,1,0,1,0,0,0,1,...,1,0,0,0,1,0,1,0,1,0
8,1000,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,1
9,1001,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,1,0


In [9]:
print_table(fault_table)

['-', 'w1_s0', 'w1_s1', 'w2_s0', 'w2_s1', 'w3_s0', 'w3_s1', 'w4_s0', 'w4_s1', 'w5_s0', 'w5_s1', 'w5_1_s0', 'w5_1_s1', 'w5_2_s0', 'w5_2_s1', 'w6_s0', 'w6_s1', 'w7_s0', 'w7_s1', 'w8_s0', 'w8_s1']
['0000', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
['0001', 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
['0010', 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
['0011', 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
['0100', 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
['0101', 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
['0110', 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
['0111', 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1000', 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
['1001', 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1010', 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
['1011', 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

# Fault Table Analysis (FTA) and Fault Collapsing

In [10]:
essential_tests, new_fault_table = prune_iterator(fault_table, find_essential, verbose=1)

['0000', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
['0001', 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
['0010', 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
['0011', 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
['0100', 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
['0101', 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
['0110', 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
['0111', 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1000', 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
['1001', 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1010', 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
['1011', 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1100', 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
['1101', 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1110', 0, 0, 1, 0, 1, 0, 0, 0, 1

In [11]:
needed_tests, new_fault_table = prune_iterator(new_fault_table, find_needed, verbose=1)

['0000', 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
['0001', 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
['0010', 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
['0011', 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
['0100', 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
['0101', 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]
['0110', 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
['0111', 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1000', 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
['1001', 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1010', 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
['1011', 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1100', 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]
['1101', 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0]
['1110', 0, 0, 1, 0, 1, 0, 0, 0, 1

In [12]:
fault_discovery_set = set()

In [13]:
print("Essential Tests")
for essential_test in essential_tests:
    faults = all_discovered_faults[input_values.index(essential_test)]
    faults = sorted(faults, key=fault_sorter)
    fault_discovery_set = fault_discovery_set.union(faults)
    print(f"{essential_test}: {faults}")

Essential Tests


In [14]:
print("Needed Tests")
for needed_test in needed_tests:
    faults = all_discovered_faults[input_values.index(needed_test)]
    faults = sorted(faults, key=fault_sorter)
    fault_discovery_set = fault_discovery_set.union(faults)
    print(f"{needed_test}: {faults}")

Needed Tests
0110: ['w2_s0', 'w3_s0', 'w5_s0', 'w5_1_s0', 'w5_2_s0', 'w6_s0', 'w7_s0', 'w8_s0']
0011: ['w1_s1', 'w2_s1', 'w5_s1', 'w5_1_s1', 'w6_s1', 'w8_s1']
1100: ['w3_s1', 'w4_s1', 'w5_s1', 'w5_2_s1', 'w7_s1', 'w8_s1']
1001: ['w1_s0', 'w4_s0', 'w6_s0', 'w7_s0', 'w8_s0']


In [15]:
n_total_faults = len(wires) * 2
fault_coverage = len(fault_discovery_set) / n_total_faults
print(f"Fault Coverage: {fault_coverage * 100} %")

Fault Coverage: 100.0 %
